<a href="https://colab.research.google.com/github/neurologic/MotorSystems_BIOL358_SP22/blob/main/NotebookColab_EMG_MultiChannel_SpikeRecorderWav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of Contents
1. [Introduction](#scrollTo=850eb426-07d8-4407-82f1-f163a2c960b6)
2. [Initialize Notebook](#scrollTo=GsbjCNcAw9VA)
3. [Import Data](#scrollTo=D2GAxldFiZaV)
4. [Select Movement](#scrollTo=jE6GzivbzjVP)
5. [Calculate EMG envelope](#scrollTo=0d1f1545-2f25-4bfe-be34-1dfb7ca3444e)
6. [Analyze Muscle Coordination](#scrollTo=84f7e282-b954-42bd-8ea2-b5edb134ec99)
7. [Connect Muscles to Movement](#scrollTo=0d494468-1f12-4f71-9196-9a77e1633a38)

<hr>

## 1. Introduction
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

By thoughtfully working through the analysis in this notebook, you will learn more about what it means for neural/muscle activity to be coordinated. 
Some of the terms and analytic techniques that you will be using:
- EMG
- amplitude envelope
- correlation

Throughout the notebook, you will plot both raw and processed data.
- You can interact with the plots by zooming in and panning. <br>
- You can save the current plot view at any time by hitting the "download" icon - it will save to your Downloads folder. Make sure to re-name the auto-generated file and make notes about what you plotted right away. <br>

To aquire this dataset, differential emg electrodes were used to simultaneously record emg activity from different muscles 
<br>
<br>






## 2. Initialize notebook.
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))


In [ ]:
#@markdown **TASK:** Run this code cell to upgrade colab packages
#@markdown >IMPORTANT: Hit the "RESTART RUNTIME" button that appears after the code cell finishes executing.

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
# import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import csv
from scipy.io import wavfile
from scipy.signal import hilbert,medfilt,resample
from sklearn.decomposition import PCA
import scipy
import seaborn as sns
from datetime import datetime,timezone,timedelta
pal = sns.color_palette(n_colors=15)
pal = pal.as_hex()


!pip install --upgrade tables

In [ ]:
#@markdown **TASK:** Run this code cell to mount your Google Drive.

from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#@markdown <b>TASK: </b> Specify the file path to some recorded data in the content folder of Colab:
#@markdown >NOTE: the data must be a .wav file (use VLCmedia player to convert if necessary)
filepath = "/content/drive/MyDrive/Classroom/BIOL358: Motor Systems/Data/Nathan_MotorUnitTypes_BYB_Recording_2022-02-14_08.24.47.wav"  #@param 
#@markdown <b>TASK: </b> After you have copied/typed in a filepath, <b> hit the play button to RUN this cell </b>


# data_folder = "/content/drive/Shareddrives/BIOL358/Data/Interossei_1chan_MotorUnitIsolation"

emg_path = Path(filepath)
print('Tasks completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))


# 3. Import Data


In [ ]:
#@markdown **TASK:** Run this code cell to load the dataset you will explore and analyze.
#@markdown This will also produce a plot of the raw signal from each channel

# datafolder = "/content/drive/Shareddrives/BIOL358/Data/Interossei_2chan_WithVideo"
# emg_file = "emg_Resampled_2022-01-10T11_39_23.csv"
# emg_path = Path(datafolder) / emg_file
sample_rate_OG, samples_OG = wavfile.read(emg_path)
# if len(np.shape(samples_OG))>1: #stereo wav file
#   samples_OG = samples_OG[:,0] #just the first channel

sample_rate = 5000 #sample rate you want to downsample to
dur = len(samples_OG)/sample_rate_OG #duration of original (to get actual sample rate)
stepfactor = int(sample_rate_OG/sample_rate) #conversion factor to step from high to low sample rate
if len(np.shape(samples_OG)) == 1:
  samples = samples_OG[0:-1:stepfactor].reshape(-1,1)
if len(np.shape(samples_OG)) > 1:
  samples = []
  for chan in samples_OG.T:
    samples.append(chan[0:-1:stepfactor]) #use conversion factor to downsample
  samples = np.asarray(samples).T
sample_rate = int(len(samples)/dur) # calculate actual new sample rate
# print(sample_rate)
xtime = np.linspace(0,dur,len(samples))

# filtert = int(0.01*sample_rate)
# y = np.abs(samples) #takes the absolute value of 
# samples_envelope = scipy.ndimage.gaussian_filter(y,filtert)
# samples_envelope =(samples_envelope - np.mean(samples_envelope)) / np.std(samples_envelope)

print('Data upload completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

print('Now be a bit patient while it plots.')
fig = go.Figure()
for i,chan in enumerate(samples.T):
  fig.add_trace(go.Scatter(x = xtime, y = chan+(i*4000),
                           opacity=0.75,name='channel' + str(i)))
# fig.add_trace(go.Scatter(x = xtime, y = samples[:,1],line_color='red',name='emg1'))
fig.update_layout(xaxis_title="time(seconds)", yaxis_title='amplitude',width=800, height=500)

## 4. Analyze a selection of the data. 
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

In [ ]:
#@markdown Pick a time window to analyze that corresponds to 
#@markdown which movement you want to analyze 
#@markdown and enter it in the code cell below.
#@markdown > Note: get the time by hovering over the data plot. 
#@markdown - **start** = the start time of the bout you want to analyze (seconds).
#@markdown - **stop** = the stop time of the bout you want to analyze (seconds).

start = None #@param {type:"number"}
stop = None #@param {type:"number"}

#@markdown <b>Task:</b> After you have specified the start and stop times,
#@markdown run this cell to execute the variable assignment.
samples_selection = samples[((xtime>start) & (xtime<stop)),:]
xtime_selection = xtime[((xtime>start) & (xtime<stop))]
print('all set - analysis domain defined')
print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## 5. Process the raw EMG signal into a rectified, smoothed "amplitude envelope"
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

This is a type of *signal processing* that transforms raw EMG activity (a bipolar signal - both positive and negative values) into what we refer to as the *amplitude envelope* of the EMG activity. The "smoothing" of the envelope results in a waveform that reflects both the amplitude and the rate of the raw EMG signal (accounts for both spatial and temporal summation). 

In [ ]:
#@markdown <b>Task:</b> Run this cell to compute the envelope of the 
#@markdown raw EMG signal for the specified time window
#@markdown and plot of the result. 
#@markdown > Note: pick a width of the temporal smoothing filter

filter_width = 0.025 #@param

fs = sample_rate
fig = go.Figure()
# Use rectfication and gaussian smoothing on EMG to get mean-centered rate
df_envelope = pd.DataFrame({})
filtert = int(filter_width*fs)
for i,chan in enumerate(samples_selection.T): # rename headers as input channels
    y = chan - np.mean(chan)
    y = np.abs(y) #takes the absolute value of 
    y = scipy.ndimage.gaussian_filter(y,filtert)
    df_envelope[i] = y
    fig.add_trace(go.Scatter(x = xtime_selection, y = y,
                             name='channel'+str(i)))

# df_rate = df_rate.subtract(df_rate.mean())
# df_rate =(df_rate - df_rate.mean()) / df_rate.std()
df_envelope['time']=xtime_selection
# df_rate = df_rate[((df_rate['time']>start) & (df_rate['time']<stop))]



# fig.add_trace(go.Scatter(x = df_rate['time'].values, y = df_rate['emg1'].values,line_color='purple',name='medial'))
fig.update_layout(xaxis_title="time(seconds)", 
                  yaxis_title='amplitude',width=800, height=400)
# fig.layout.xaxis.showticklabels=False
print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))
fig.show()

## 6. Muscle coordination
([Return to Table of Contents](#scrollTo=zIt8pCksxZic))

Think about how you would qualitatively describe the relationship between the two muscle signals that you plotted. The correlation metric is often helpful to quantify the relationship between signals. Essentially, correlation is the measure of how two or more variables are related to one another. https://en.wikipedia.org/wiki/Correlation



In [ ]:
#@markdown <b>Task:</b> Run this cell to: <br> 1. calculate the 
#@markdown mathematical correlation values between the two signals and
#@markdown <br> 2. plot the lateral and medial EMG envelopes against each other
#@markdown (rather than across time as you did before). 

#@markdown Think about how the scatterplot is a way to visualize *correlation*.
#@markdown Does the scatterplot change the way you interpret the correlation value that you calculated?

# No need to edit this code cell
################################
print('correlation matrix:')
p_corr = df_rate.drop('time',axis=1).corr()
# hfig, ax = plt.subplots(1)
# sns.heatmap(p_corr, annot=True)
display(p_corr)

fig = go.Figure()
fig.add_trace(go.Scatter(x = df_rate['emg0'].values, 
                         y = df_rate['emg1'].values,
                        #  marker_color='black',
                         mode = 'markers', 
                         marker=dict(
                          color='black',
                          size=2,
                          opacity=0.5,)))
fig.update_layout(xaxis_title="amplitude lateral interossei", 
                  yaxis_title='amplitude medial interossei',
                  width=400, height=400)
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
# axs.set_ylabel('amplitude (a.u.) of your second EMG channel')
# axs.set_xlabel('amplitude (a.u.) of your first EMG channel');
